# joint student loans + joint accounts in general

does the Equifax data have a flag for whether a student loan is jointly held? And would we know who is the joint holder? I wonder if we can find out if a parent co-holds the debt of a child.

Also looks at how many joint accounts there are for each kind of debt. 

In [1]:
# Setting up 

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

sample = "~//projects//equifaxmacro_proj//EquiFax3//2_10_Percent_Sample_Construction//output//sample//"
biao = "~//projects//equifaxmacro_proj//EquiFax3//3_Balance_In_And_Out_Construction//output//"

In [2]:
cols = ['consumer_id', 'trade_id', 'product_category', 'ecoa', 'balance', 'consumer_age_archive']

mon_str = sample + "202005_10perc.parquet"
mon = pd.read_parquet(mon_str, columns = cols)

In [47]:
sl = mon[mon.product_category.isin(['SL1', 'SL2'])]

In [48]:
sl.ecoa.value_counts()

I    1623379
J      97211
C      22115
M      20032
X        626
T        360
S          3
U          1
Name: ecoa, dtype: int64

In [44]:
1623379/1763729

0.9204242828688535

In [36]:
sl_j = sl[sl.ecoa!= "I"]

In [37]:
sl_ids = sl_j.trade_id.value_counts()
sl_ids = sl_ids[sl_ids >= 2].index

In [38]:
sl_ids

Int64Index([ 946094903,  493555205, 1025660910,  996730568,  861478639,
             984693587,  968007179,  908733856,  979700326,  965909220,
            ...
             819617605, 1050681540,  774428633, 1030958533,  987701092,
             818975398,  819193235,  814450139,  988850970,  984344124],
           dtype='int64', length=411)

In [39]:
two_ids = sl_j[sl_j.trade_id.isin(sl_ids)].sort_values('trade_id')

In [40]:
two_ids.ecoa.value_counts()

J    464
C    187
M    171
Name: ecoa, dtype: int64

In [56]:
two_ids.tail(10)

,consumer_id,trade_id,product_category,ecoa,balance,consumer_age_archive
16002917,32558716,1045658751,SL2,J,6034,1
16002918,11786553,1045658751,SL2,J,6034,5
6224690,12060739,1047011475,SL1,J,2495,5
6224689,32110469,1047011475,SL1,J,2495,1
7053585,1920961,1047540686,SL1,C,15395,5
7053584,32534834,1047540686,SL1,M,15395,1
25060221,7837518,1050235788,SL2,C,9514,5
25060222,31687525,1050235788,SL2,M,9514,1
5657005,3828172,1050681540,SL2,J,4833,5
5657006,27201004,1050681540,SL2,J,4833,2


## What percentage of each kind of loan is joint? 

Do I want to use balances in and out to track this or do it within the loop? 

Let's do it within the loop actually--that way we can track differences over time. And if nothing changes, then it'll be easy to track percentages

In [3]:
# number of tradelines
tl = mon.groupby('product_category').count()

# number of joint tradelines 
jtl = mon[mon.ecoa=="J"].groupby('product_category').count()

In [4]:
jtl/tl

,consumer_id,trade_id,ecoa,balance,consumer_age_archive
product_category,,,,,
AB1,0.321224,0.321224,0.321224,0.321224,0.321224
AB2,0.423530,0.423530,0.423543,0.423530,0.423530
AF1,0.366390,0.366390,0.366398,0.366390,0.366390
AF2,0.425026,0.425026,0.425043,0.425026,0.425026
BC,0.063076,0.063076,0.063080,0.063076,0.063076
CFI,0.126383,0.126383,0.126557,0.126383,0.126383
CFR,0.093782,0.093782,0.093782,0.093782,0.093782
FM,0.641503,0.641503,0.641807,0.641503,0.641503
HI,0.513582,0.513582,0.513746,0.513582,0.513582
